<a href="https://colab.research.google.com/github/Yuan-Chun-Chih/HW_5/blob/main/HW_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
               gradio pandas beautifulsoup4 google-generativeai python-dateutil folium -q

import requests
import pandas as pd
import gradio as gr
import asyncio
import random
from google.colab import auth
import gspread
from google.auth import default
from google.colab import userdata
import google.generativeai as genai
import folium

In [3]:
# =====================================================
# 1️⃣ 取得 OSM 餐廳資料
# =====================================================
def fetch_restaurants(lat=25.037516, lon=121.5637, radius=1000):
    """使用 Overpass API 抓取指定範圍內的餐廳資料"""
    query = f"""
    [out:json];
    node
      [amenity=restaurant]
      (around:{radius},{lat},{lon});
    out;
    """
    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data={'data': query})

    if response.status_code != 200:
        print(f"⚠️ Error fetching data: Status code {response.status_code}")
        print(f"Response text: {response.text}")
        return pd.DataFrame()  # Return an empty DataFrame on error

    try:
        data = response.json()
    except requests.exceptions.JSONDecodeError as e:
        print(f"⚠️ Error decoding JSON: {e}")
        print(f"Response text: {response.text}")
        return pd.DataFrame()

    restaurants = []
    for el in data.get("elements", []):
        tags = el.get("tags", {})
        restaurants.append({
            "餐廳名稱": tags.get("name", "未命名"),
            "餐飲類型": tags.get("cuisine", "未知"),
            "地址": tags.get("addr:street", "無地址"),
            "價位": random.choice(['$', '$$', '$$$']),
            "距離": random.choice(['短', '中', '長']),
            "是否營業": random.choice(['是', '否']),
            "特色描述": random.choice(['道地台味', '氣氛佳', '高CP值', '快速方便', '份量足']),
            "經度": el.get("lon"),
            "緯度": el.get("lat")
        })

    df = pd.DataFrame(restaurants)
    print(f"✅ 找到 {len(df)} 家餐廳")
    return df

df_restaurants = fetch_restaurants()

✅ 找到 504 家餐廳


In [5]:
# =====================================================
# 2️⃣ 設定 Gemini AI
# =====================================================
api_key = userdata.get('gemini')
if api_key:
    genai.configure(api_key=api_key)
    model = genai.GenerativeModel('gemini-2.5-pro')
else:
    print("⚠️ 未設定 Gemini API 金鑰，將使用模擬回覆。")

In [6]:
# =====================================================
# 3️⃣ 篩選與推薦函數
# =====================================================
def filter_and_recommend(df, price_filter, distance_filter, is_available):
    price_list = price_filter if price_filter else df['價位'].unique()
    distance_list = distance_filter if distance_filter else df['距離'].unique()
    available_status = ['是'] if is_available else ['是', '否']

    filtered_df = df[
        df['價位'].isin(price_list) &
        df['距離'].isin(distance_list) &
        df['是否營業'].isin(available_status)
    ].copy()

    if filtered_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", ""

    n_recommend = min(3, len(filtered_df))
    recommended_df = filtered_df.sample(n=n_recommend, replace=False)

    recommend_list = "\n".join([
        f"{i+1}. {row['餐廳名稱']} ({row['價位']} / {row['距離']}) - {row['特色描述']}"
        for i, row in recommended_df.iterrows()
    ])

    ai_input = "、".join([
        f"{row['餐廳名稱']}，價位{row['價位']}，距離{row['距離']}，特色：{row['特色描述']}"
        for _, row in recommended_df.iterrows()
    ])

    return recommended_df, recommend_list, ai_input


async def generate_summary(ai_input):
    if not ai_input:
        return "請先點擊「開始推薦」以產生餐廳清單。"
    await asyncio.sleep(1)
    return f"AI 助理建議：今天可考慮 {ai_input.split('、')[0]}，快速又實惠；或試試 {ai_input.split('、')[1]}，氣氛更好。"


async def wrapper_recommend(price_filter, distance_filter, is_available):
    recommended_df, recommend_list, ai_input = filter_and_recommend(
        df_restaurants, price_filter, distance_filter, is_available
    )
    ai_summary = await generate_summary(ai_input)
    display_df = recommended_df[['餐廳名稱', '價位', '距離']].reset_index(drop=True)

    if display_df.empty:
        return pd.DataFrame(), "沒有找到符合條件的餐廳。", "（無建議）"
    return display_df, recommend_list, ai_summary

In [7]:
# =====================================================
# 4️⃣ Gradio 互動介面
# =====================================================
with gr.Blocks(title="午餐決策器 (無金鑰版)") as demo:
    gr.Markdown("""
    # 🍜 午餐決策器 (免 Google Maps Key)
    使用 OpenStreetMap 資料自動推薦附近餐廳！
    """)

    with gr.Row():
        with gr.Column(scale=1):
            price_select = gr.CheckboxGroup(['$', '$$', '$$$'], label="價位", value=['$', '$$'])
            distance_select = gr.CheckboxGroup(['短', '中', '長'], label="距離", value=['短', '中'])
            available_toggle = gr.Checkbox(label="只看營業中", value=True)
            recommend_btn = gr.Button("🚀 開始推薦")

        with gr.Column(scale=2):
            output_dataframe = gr.DataFrame(headers=["餐廳名稱", "價位", "距離"], datatype=["str", "str", "str"])
            output_list = gr.Textbox(label="推薦清單")

    output_summary = gr.Textbox(label="AI 決策建議", interactive=False)

    recommend_btn.click(
        fn=wrapper_recommend,
        inputs=[price_select, distance_select, available_toggle],
        outputs=[output_dataframe, output_list, output_summary]
    )

    gr.Markdown("---")
    gr.Markdown("本系統使用 Overpass API + OpenStreetMap，完全免 Google Maps API Key。")

if __name__ == "__main__":
    print("Gradio 應用啟動中...")
    demo.launch(inbrowser=True)

Gradio 應用啟動中...
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://674de95bb86593fed3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
